# 3 test business logic

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

## confirm the `send_cmd` command

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### publish a MQTT message to emulate gamepad action

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-type: application/json"  http://${HOST_IPADDR}:8080/controller/api/v1/shipments/ -X POST -d @-<<__EOS__
{
  "destination": {
    "name": "LICTiA管理室"
  },
  "updated": [
    {
      "prev_quantity": 5,
      "new_quantity": 4,
      "reservation": 1,
      "title": "美味しい天然水（2L）",
      "place": "ホワイトボード室"
    },
    {
      "prev_quantity": 6,
      "new_quantity": 4,
      "reservation": 2,
      "title": "美味しい天然水（2L）",
      "place": "会議室1"
    },
    {
      "prev_quantity": 3,
      "new_quantity": 2,
      "reservation": 1,
      "title": "美味しいメロンパン",
      "place": "ホワイトボード室"
    }
  ]
}
__EOS__

example)
```
HTTP/1.1 201 Created
server: envoy
date: Wed, 07 Aug 2019 08:18:19 GMT
content-type: application/json
content-length: 21
x-envoy-upstream-service-time: 66

{"result":"success"}
```

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/63ZwU16FdHZ67aPkHS received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmd', ... (128 bytes))
{"send_cmd":{"time":"2019-08-07T17:18:19.402+09:00","cmd":"navi","waypoints":[{"x":0.1,"y":0.2,"z":0},{"x":1.1,"y":0.2,"z":0}]}}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` is `PENDING`

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-07T08:18:19.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": [
      [
        {
          "x": 0.1,
          "y": 0.2,
          "z": 0
        },
        {
          "x": 1.1,
          "y": 0.2,
          "z": 0
        }
      ],
      [
        {
          "x": 1.1,
          "y": 1.5,
          "z": 0
        }
      ],
      [
        {
          "x": 2.1,
          "y": 2,
          "z": 0
        }
      ],
      [
        {
          "x": 1.1,
          "y": 1.5,
          "z": 0
        },
        {
          "x": 1.1,
          "y": 0.2,
          "z": 0
        },
        {
          "x": 0.1,
          "y": 0.2,
          "z": 0
        },
        {
          "x": 0,
          "y": 0,
          "z": 0
        }
      ]
    ],
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-08-07T17:18:19.402+09:00"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": {
      "time": "2019-08-07T16:43:16.846477+09:00",
      "received_time": "2019-07-05T07:28:26.1562279306+09:00",
      "received_cmd": "navi",
      "received_waypoints": [
        {
          "x": "0.503",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "2.81",
          "z": "0"
        }
      ],
      "result": "ack",
      "errors": ""
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:43:16.999Z"
      }
    }
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T08:18:19.425Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-07T16:37:16.323513+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving command result

In [ ]:
m=$(cat << __EOS__ 
{
  "send_cmd": {
    "time": "$(now)",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "x": 0.1,
        "y": 0.2,
        "z": 0
      },
      {
        "x": 1.1,
        "y": 0.2,
        "z": 0
      }
    ],
    "result": "ack",
    "errors": []
  }
}
__EOS__
)
mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/cmdexe -m "${m}"

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/63ZwU16FdHZ67aPkHS received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmdexe', ... (368 bytes))
{
  "send_cmd": {
    "time": "2019-08-07T17:21:25.140495+09:00",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "x": 0.1,
        "y": 0.2,
        "z": 0
      },
      {
        "x": 1.1,
        "y": 0.2,
        "z": 0
      }
    ],
    "result": "ack",
    "errors": []
  }
}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` turns to `OK` and `send_cmd_info` has the result message

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-07T08:21:25.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": [
      [
        {
          "x": 0.1,
          "y": 0.2,
          "z": 0
        },
        {
          "x": 1.1,
          "y": 0.2,
          "z": 0
        }
      ],
      [
        {
          "x": 1.1,
          "y": 1.5,
          "z": 0
        }
      ],
      [
        {
          "x": 2.1,
          "y": 2,
          "z": 0
        }
      ],
      [
        {
          "x": 1.1,
          "y": 1.5,
          "z": 0
        },
        {
          "x": 1.1,
          "y": 0.2,
          "z": 0
        },
        {
          "x": 0.1,
          "y": 0.2,
          "z": 0
        },
        {
          "x": 0,
          "y": 0,
          "z": 0
        }
      ]
    ],
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-08-07T17:18:19.402+09:00"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": {
      "time": "2019-08-07T17:21:25.140495+09:00",
      "received_time": "2019-07-05T07:28:26.1562279306+09:00",
      "received_cmd": "navi",
      "received_waypoints": [
        {
          "x": "0.1",
          "y": "0.2",
          "z": "0"
        },
        {
          "x": "1.1",
          "y": "0.2",
          "z": "0"
        }
      ],
      "result": "ack",
      "errors": ""
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T08:21:25.379Z"
      }
    }
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T08:21:25.379Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-07T16:37:16.323513+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
c=$(echo sth_${DELIVERY_ROBOT_SERVICEPATH}xffff${DELIVERY_ROBOT_01}xffff${DELIVERY_ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"${c}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/sth_uoa_poc2?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("d03a184b-19e0-4415-a0d3-81ba0a1d2b29") }
MongoDB server version: 4.1.13
{ "_id" : ObjectId("5d4a8a061283940016d8af4d"), "recvTime" : ISODate("2019-08-07T08:21:25.379Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "OK" }
{ "_id" : ObjectId("5d4a8a061283940016d8af4e"), "recvTime" : ISODate("2019-08-07T08:21:25.379Z"), "attrName" : "send_cmd_info", "attrType" : "commandResult", "attrValue" : "{\"time\":\"2019-08-07T17:21:25.140495+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.1\",\"y\":\"0.2\",\"z\":\"0\"},{\"x\":\"1.1\",\"y\":\"0.2\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}" }
{ "_id" : ObjectId("5d4a894b8f7d1300163a452d"), "recvTime" : ISODate("2019-08-07T08:18:19.425Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5d4a894b0c76920016cabc54"), "recvTime" : ISODate("2019-08-07T08:18:19.402Z"), "attrName" : "current_routes", "attrType" : "array", "attrValue" : "[[{\"x\":0.1,\"y\":0.2,\"z\":0},{\"x\":1.1,\"y\":0.2,\"z\":0}],[{\"x\":1.1,\"y\":1.5,\"z\":0}],[{\"x\":2.1,\"y\":2,\"z\":0}],[{\"x\":1.1,\"y\":1.5,\"z\":0},{\"x\":1.1,\"y\":0.2,\"z\":0},{\"x\":0.1,\"y\":0.2,\"z\":0},{\"x\":0,\"y\":0,\"z\":0}]]" }
...
```

### confirm sth-comet

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
for a in "send_cmd" "send_cmd_status" "send_cmd_info" "current_routes"; do
  curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/comet/STH/v1/contextEntities/type/${DELIVERY_ROBOT_TYPE}/id/${DELIVERY_ROBOT_01}/attributes/${a}?lastN=100 | jq .
done

example)
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd",
            "values": []
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd_status",
            "values": [
              {
                "recvTime": "2019-08-07T07:24:59.402Z",
                "attrType": "commandStatus",
                "attrValue": "UNKNOWN"
              },
              {
                "recvTime": "2019-08-07T07:39:34.172Z",
                "attrType": "commandStatus",
                "attrValue": "PENDING"
              },
              {
                "recvTime": "2019-08-07T07:43:16.999Z",
                "attrType": "commandStatus",
                "attrValue": "OK"
              },
              {
                "recvTime": "2019-08-07T08:18:19.425Z",
                "attrType": "commandStatus",
                "attrValue": "PENDING"
              },
              {
                "recvTime": "2019-08-07T08:21:25.379Z",
                "attrType": "commandStatus",
                "attrValue": "OK"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd_info",
            "values": [
              {
                "recvTime": "2019-08-07T07:43:16.999Z",
                "attrType": "commandResult",
                "attrValue": "{\"time\":\"2019-08-07T16:43:16.846477+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}"
              },
              {
                "recvTime": "2019-08-07T07:43:16.999Z",
                "attrType": "commandResult",
                "attrValue": "{\"time\":\"2019-08-07T16:43:16.846477+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}"
              },
              {
                "recvTime": "2019-08-07T07:43:16.999Z",
                "attrType": "commandResult",
                "attrValue": "{\"time\":\"2019-08-07T16:43:16.846477+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}"
              },
              {
                "recvTime": "2019-08-07T08:21:25.379Z",
                "attrType": "commandResult",
                "attrValue": "{\"time\":\"2019-08-07T17:21:25.140495+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.1\",\"y\":\"0.2\",\"z\":\"0\"},{\"x\":\"1.1\",\"y\":\"0.2\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "current_routes",
            "values": [
              {
                "recvTime": "2019-08-07T08:18:19.402Z",
                "attrType": "array",
                "attrValue": "[[{\"x\":0.1,\"y\":0.2,\"z\":0},{\"x\":1.1,\"y\":0.2,\"z\":0}],[{\"x\":1.1,\"y\":1.5,\"z\":0}],[{\"x\":2.1,\"y\":2,\"z\":0}],[{\"x\":1.1,\"y\":1.5,\"z\":0},{\"x\":1.1,\"y\":0.2,\"z\":0},{\"x\":0.1,\"y\":0.2,\"z\":0},{\"x\":0,\"y\":0,\"z\":0}]]"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```